# Components generator

### Parameters

In [2]:
import math 
import numpy as np

In [3]:
#########################################
# Setup parameters
#########################################

N = 4                   # Input size (N x N)
TAPS = 6                # Number of taps
FILTERS = 15            # Number of filters (= number of outputs expected)
# OUTPUT_WIDTH = [11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11]         # Bitwidth for each output from interpolator
OUTPUT_WIDTH = 11       # One size for every output, as it is for my case

### Register

In [4]:
#########################################
# Variables calulation: Register
#########################################

number_of_regs = N + TAPS - 1
one_reg_size = FILTERS * OUTPUT_WIDTH

In [5]:
#########################################
# Text writing: One register
#########################################

register_text = '''/*------------------------------------------------------------------------------
 * File: reg_buffer_'''+str(one_reg_size)+'''.v
 * Date generated: 13/03/2023
 * Date modified: 21/03/2023
 * Author: Bruna Suemi Nagai
 * Description: Module of one register as part of the buffer.
 *------------------------------------------------------------------------------ */

module reg_buffer_'''+str(one_reg_size)+'''(
    input clk,
    input reset,
    input enable,
    input ['''+str(one_reg_size-1)+''':0] data_in,
    output reg ['''+str(one_reg_size-1)+''':0] data_out
);

always @(posedge clk, negedge reset) begin
    if (!reset) begin
        data_out <= '''+str(one_reg_size)+''''b0;
    end else if (enable) begin
        data_out <= data_in;
    end
end

endmodule
'''

print(register_text)

/*------------------------------------------------------------------------------
 * File: reg_buffer_165.v
 * Date generated: 13/03/2023
 * Date modified: 21/03/2023
 * Author: Bruna Suemi Nagai
 * Description: Module of one register as part of the buffer.
 *------------------------------------------------------------------------------ */

module reg_buffer_165(
    input clk,
    input reset,
    input enable,
    input [164:0] data_in,
    output reg [164:0] data_out
);

always @(posedge clk, negedge reset) begin
    if (!reset) begin
        data_out <= 165'b0;
    end else if (enable) begin
        data_out <= data_in;
    end
end

endmodule

